In [49]:
%config IPCompleter.greedy=True
from sklearn.feature_extraction.text import CountVectorizer
from nltk.translate.bleu_score import corpus_bleu
from nltk.translate.bleu_score import sentence_bleu
import json
import re

In [62]:
#load vocabulary
with open('data/vocabulary.json', 'r') as inf:
    vocabulary = json.load(inf) 
vectorizer = CountVectorizer(vocabulary=vocabulary, token_pattern=r'\S+', stop_words=['<nl>'])

analyzer = vectorizer.build_analyzer()

In [63]:
#calculates bleu_score
candidates = []
with open('data/generated.msg', 'r') as inf:
    for msg in inf:
        cur_candidate = analyzer(msg)
        candidates.append(cur_candidate)
        
references = []
with open('data/test.msg', 'r') as inf:
    for msg in inf:
        cur_reference = analyzer(msg)
        references.append([cur_reference])

In [64]:
print(len(candidates))
print(len(references))

12601
12601


In [65]:

        
corpus_bleu_score = corpus_bleu(references, candidates)
corpus_bleu_score

0.0026756369053088556

In [66]:
#evaluates tags prediction
generated_tags = []
with open('data/generated.msg', 'r') as inf:
    for line in inf:
        tag = re.match(r'\[(.*?)\]', line)
        tag = tag.group(1) if tag is not None else ''
        generated_tags.append(tag)
        
test_tags = []
with open('data/test.msg', 'r') as inf:
    for line in inf:
        tag = re.match(r'\[(.*?)\]', line)
        tag = tag.group(1) if tag is not None else ''
        test_tags.append(tag)
        
count_test_tags = 0
right_test_tags = 0

for i in range(len(test_tags)):
    if test_tags[i] and generated_tags[i]:
        count_test_tags += 1
        if test_tags[i] == generated_tags[i]:
            right_test_tags += 1
        
tag_precision = right_test_tags / count_test_tags
print(f'{right_test_tags} right tags of {count_test_tags}, precision is {tag_precision}')

687 right tags of 1234, precision is 0.5567260940032415


In [22]:
#bleu_score for specialy generated best possible data
candidates = []
with open('data/pseudo_generated.msg', 'r') as inf:
    for msg in inf:
        cur_candidate = analyzer(msg)
        candidates.append(cur_candidate)
        
references = []
with open('data/test.msg', 'r') as inf:
    for msg in inf:
        cur_reference = analyzer(msg)
        references.append([cur_reference])
        
corpus_bleu_score = corpus_bleu(references, candidates)
corpus_bleu_score

0.37379595849191893

In [48]:
candidates = []
dataset = 'data'
with open('{dataset:}/generated.msg'.format(dataset=dataset), 'r') as inf:
    for msg in inf:
        cur_candidate = analyzer(msg)
        candidates.append(cur_candidate)
        
references = []
with open('{dataset:}/test.msg'.format(dataset=dataset), 'r') as inf:
    for msg in inf:
        cur_reference = analyzer(msg)
        references.append([cur_reference])

with open('{dataset:}/stats.txt'.format(dataset=dataset), 'w') as ouf: 
    ouf.write('BLEU score: {}\n\n'.format(corpus_bleu_score))
    n = len(candidates)
    bleu_sum = 0
    print(references[0], candidates[0])
    cnt = 0
    for i in range(n):
        bleu_score = sentence_bleu(references[i], candidates[i])
        if (bleu_score > 0.005):
            cnt += 1
            ouf.write(' '.join(references[i][0]) + '\n')
            ouf.write(' '.join(candidates[i]) + '\n')
            ouf.write('\n')
    ouf.write('{} good of {} total ({:.0f}%)\n'.format(cnt, n, cnt * 100 / n))

[['[skeletons]', 'fix', 'sanitizing', 'results', 'of', 'repr', '(', ')', 'on', 'attribute', 'values']] ['{py}', 'we', 'should', 'show', 'python', 'banner', 'even', 'if', 'starting', 'script', 'in', 'python', 'console', 'is', 'empty']
